# FAA Aircraft Registration Database

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests
import matplotlib as mpl
import geopandas as gpd
import json
import numpy as np
from skimpy import clean_columns

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [4]:
# https://www.faa.gov/licenses_certificates/aircraft_certification/aircraft_registry/releasable_aircraft_download/

In [5]:
# https://registry.faa.gov/database/ReleasableAircraft.zip

### Read master and reference files

In [12]:
master = pd.read_csv(
    "/Users/stiles/data/aircraft/ReleasableAircraft/MASTER.txt", low_memory=False
)

In [13]:
reference = pd.read_csv(
    "/Users/stiles/data/aircraft/ReleasableAircraft/ACFTREF.txt", low_memory=False
)

### Clean up columns

In [ ]:
master.columns = (
    master.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_", regex=False)
    .str.replace("(", "", regex=False)
    .str.replace(")", "", regex=False)
    .str.replace("-", "_", regex=False)
)

In [ ]:
reference.columns = (
    master.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_", regex=False)
    .str.replace("(", "", regex=False)
    .str.replace(")", "", regex=False)
    .str.replace("-", "_", regex=False)
)

In [ ]:
reference.loc[(reference["type_acft"] == "4"), "type"] = "Fixed wing single engine"
reference.loc[(reference["type_acft"] == "5"), "type"] = "Fixed wing multi engine"
reference.loc[(reference["type_acft"] == "6"), "type"] = "Rotorcraft"

In [ ]:
master.name = master.name.str.strip()
master.certification = master.certification.str.strip()
master.type_registrant = master.type_registrant.str.strip()

### Owners

In [ ]:
owners = (
    master.groupby(["type_registrant", "certification"])
    .agg("size")
    .reset_index(name="planes")
)

In [ ]:
owners.sort_values("planes", ascending=False).head(10)

In [ ]:
corptrans = owners[(owners["planes"] > 20) & (owners["certification"] == "1T")]

In [ ]:
airlines = [
    "DELTA AIR LINES INC",
    "AMERICAN AIRLINES INC",
    "SKYWEST AIRLINES INC",
    "SWIFT AIR LLC",
    "UNITED AIRLINES INC",
    "SOUTHWEST AIRLINES CO",
    "JETBLUE AIRWAYS CORP",
    "HORIZON AIR INDUSTRIES",
    "ALASKA AIRLINES INC",
    "REPUBLIC AIRWAYS INC",
    "SPIRIT AIRLINES INC",
    "MESA AIRLINES INC",
]

In [ ]:
airlines

In [ ]:
cargo = [
    "UNITED PARCEL SERVICE CO",
    "FEDERAL EXPRESS CORP",
    "FEDERAL EXPRESS CORPORATION",
    "CARGO AIRCRAFT MANAGEMENT INC",
    "ATLAS AIR INC",
]

In [ ]:
cargo

In [ ]:
private = ["NETJETS SALES INC", "FLEXJET LLC"]

In [ ]:
corptrans.sort_values(by="planes", ascending=False).head(10)

In [ ]:
airline_planes = master

In [ ]:
airline_planes_slim = airline_planes[["name", "n_number"]]

In [ ]:
airline_planes_slim.to_csv("output/airline_planes.csv", index=False)

### Airline planes

In [ ]:
american = master[master["name"].str.contains("AMERICAN AIRLINES")]

In [ ]:
american.iloc[583]

In [ ]:
master["n_number"] = "N" + master["n_number"]

In [ ]:
src = master.merge(reference, left_on="mfr_mdl_code", right_on="code")

In [ ]:
src[src["n_number"] == "N302FD"]

In [ ]:
df = src.copy()

In [ ]:
len(df[~df["name"].isin(airlines) | ~df["name"].isin(cargo)])

In [ ]:
planes = df[
    (~df["name"].isin(airlines))
    & (~df["name"].isin(cargo))
    & (df["type_aircraft"].str.contains("4|5"))
    & (df["certification"].str.contains("1T"))
]

In [ ]:
len(planes)

### Export

In [ ]:
df.to_csv("/Users/stiles/data/aircraft/master_faa_owners_database.csv", index=False)